


<h1 style="color: blue; font-weight: bold; text-align: center; font-size: 24px;">Research Paper Publishability Assessment and  Conference Selection</h1>


In [ ]:
from IPython.display import display, HTML

display(HTML('''
<h1 style="color: blue; font-weight: bold; text-align: center; font-size: 48px;">
    Research Paper Publishability Assessment and Conference Selection
</h1>
'''))


#Full notebook

<h1 style="color: blue; font-weight: bold; text-align: center;">Importing necessary Libraries</h1>



##we have installed and imported  necessary libraries

In [ ]:
!pip install -U pathway[all]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 27.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.7 MB/s eta 

In [ ]:
!pip install pathway

In [ ]:
!apt-get install poppler-utils

In [ ]:
!pip install "pathway[xpack-llm-docs]"

In [ ]:
!pip install --upgrade tiktoken -q

In [ ]:
import pathway  as pw
import openparse as op
import tiktoken
from pathway.xpacks.llm.parsers import ParseUnstructured
from pathway.xpacks.llm.splitters import TokenCountSplitter

<h2 style="color: blue; font-weight: bold; text-align: center;"><h1 style="color: blue; font-weight: bold; text-align: center; font-size: 24px;">Dataset Processing </h2>


 ### importing the dataset using google drive connector and parsing it to extract data in form of text and storing it form of  dataframe.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import json

# Define the file path
file_path = '/content/drive/My Drive/KDSH/credentials.json'

# Open and load the JSON file
with open(file_path, 'r') as file:
    data = json.load(file)

In [ ]:
@pw.udf
def strip_metadata(docs:list[tuple[str, dict]]) -> list[str]:
    return [doc[0] for doc in docs]

table = pw.io.gdrive.read(
    object_id="1Z8z4craj36ighb8hzUzeM76OOgpUdsKr",
    service_user_credentials_file=file_path,
    with_metadata=True,
    mode="static"
)


parser=  ParseUnstructured()
doc= table.select(text=parser(pw.this.data))
doc=doc.select(text=strip_metadata(pw.this.text))
df=pw.debug.table_to_pandas(doc)

In [ ]:
df.to_csv('output.csv', index=False)


In [ ]:
import pandas as pd
df=pd.read_csv('output.csv')

<p style="color: black; font-weight: normal; text-align: left; font-size: 14px;">.</p>


<h2 style="color: blue; font-weight: bold; text-align: center;"><h1 style="color: blue; font-weight: bold; text-align: center; font-size: 24px;">Text Preprocessing </h2>

###Dividing in sections and text preprocessing

In [ ]:
for i in range(150):
  df['text'][i]=df['text'][i][0]

In [ ]:
import re

def structure_text(text):
    """
    Structure the text into sections based on the presence of 'Abstract'
    and the presence of a number followed by 1-2 words between line breaks.
    :param text: Raw extracted text from the PDF.
    :return: Dictionary containing structured sections.
    """
    # Replace literal \\n with actual newlines
    text = text.replace("\\n", "\n")

    sections = {}
    lines = text.splitlines()
    current_section = "Topic Name"
    sections[current_section] = ""

    # Regex to detect a line with a number followed by 1-2 words (e.g., "1 Introduction")
    section_pattern = re.compile(r"^\d+\s+[A-Za-z]+(?:\s+[A-Za-z]+)?$")  # Allow 1 or 2 words after the number
    abstract_found = False

    for line in lines:
        line = line.strip()
        if not line:
            continue

        # Detect Abstract as the first section
        if not abstract_found and "abstract" in line.lower():
            current_section = "Abstract"
            sections[current_section] = ""
            abstract_found = True
            continue

        # Check if the line matches the section pattern (e.g., "1 Introduction")
        if section_pattern.match(line):
            current_section = line
            sections[current_section] = ""
        else:
            # Append content to the current section
            sections[current_section] += line + " "

    return sections




In [ ]:
df['text_dict']=df['text'].apply(structure_text)

In [ ]:
import spacy
import string
import nltk
from nltk.corpus import stopwords

# Ensure NLTK stopwords are downloaded
nltk.download('stopwords')

def preprocess_text(text):
    """
    Preprocess the text by removing conjunctions, adverbs, adjectives, stopwords, and other irrelevant POS.

    Args:
        text (str): The input text to preprocess.

    Returns:
        str: The cleaned and preprocessed text.
    """
    # Load the spaCy English model
    nlp = spacy.load("en_core_web_sm")

    # Parse the text with spaCy
    doc = nlp(text)

    # Define POS tags to remove
    pos_to_remove = {"CCONJ", "ADV", "ADJ", "PART", "INTJ", "DET", "PRON", "AUX", "ADP", "SCONJ"}

    # Get NLTK stopwords
    stop_words = set(stopwords.words("english"))

    # Filter out tokens with unwanted POS tags, punctuation, stopwords, and digits
    filtered_tokens = [
        token.text for token in doc
        if token.pos_ not in pos_to_remove
        and token.text not in string.punctuation
        and not token.is_digit
        and token.text.lower() not in stop_words
    ]

    # Join filtered tokens to form the cleaned text
    cleaned_text = " ".join(filtered_tokens)


    return cleaned_text

In [ ]:
df['texty']=df['text'].apply(preprocess_text)

In [ ]:
import numpy as np
df['target']=np.nan

In [ ]:
for i in range(11):
  df['target'][i]=1
for i in range(11,16):
  df['target'][i]=0

In [ ]:
#Classification done by pre trained SciBERT model
out=[1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1]
for i in range(135):
  df['target'][i+15]=out[i]

NameError: name 'df' is not defined

<h2 style="color: blue; font-weight: bold; text-align: center;"><h1 style="color: blue; font-weight: bold; text-align: center; font-size: 24px;">Task1 </h2>

###Classifying in publishable or not

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import numpy as np

# Step 1: Prepare the data (example assumes df['text'] is the text column)
vectorizer = CountVectorizer(max_features=500)  # Adjust max_features as needed
X = vectorizer.fit_transform(df['text'])

# Step 2: Train LDA with 5 topics
lda = LatentDirichletAllocation(n_components=5, random_state=42)
lda.fit(X)

# Step 3: Compute the topic distributions for each document
# Instead of computing perplexity for each document, we'll extract the topic distribution
topic_distributions = lda.transform(X)  # This will give us the topic distribution for each document

# Step 4: Standardize the topic distributions for better clustering
scaler = StandardScaler()
topic_distributions_scaled = scaler.fit_transform(topic_distributions)

# Step 5: Use KMeans clustering to group documents based on the topic distributions
kmeans = KMeans(n_clusters=2, random_state=42)  # Change n_clusters as needed
clusters = kmeans.fit_predict(topic_distributions_scaled)

# Step 6: Add cluster assignments to the dataframe
df['cluster'] = clusters

# Output the results
print("Clustered DataFrame based on Topic Distributions:")
print(df[['text', 'cluster']])



In [ ]:
nlp = spacy.load("en_core_web_sm")

# Function to extract proper nouns
def extract_proper_nouns(text):
    doc = nlp(text)
    proper_nouns = [token.text for token in doc if token.pos_ == "PROPN"]
    return proper_nouns

In [ ]:
df['proper nouns']=df['texty'].apply(extract_proper_nouns)

In [ ]:
df['proper nouns']

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import KeyedVectors
from transformers import BertTokenizer, BertModel
import torch

# Function to get word vectors for a list of words
def get_word_vectors(words):
    """
    Retrieve word vectors for the list of words.

    Args:
        words (list): List of words (proper nouns).

    Returns:
        list: List of word vectors for the words.
    """
    # Choose an embedding model (e.g., Word2Vec, GloVe, or BERT)

    # Example using BERT
    model_name = 'bert-base-uncased'
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertModel.from_pretrained(model_name)

    vectors = []
    for word in words:
        inputs = tokenizer(word, return_tensors="pt")
        with torch.no_grad():
            outputs = model(**inputs)
        # Extract the vector for the word (using [CLS] token's embedding)
        word_vector = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
        vectors.append(word_vector)

    return vectors

def average_cosine_similarity(proper_nouns):
    """
    Calculate the average cosine similarity among proper nouns in the text.

    Args:
        proper_nouns (list): List of proper noun words.

    Returns:
        float: The average cosine similarity among proper nouns.
    """
    if len(proper_nouns) < 2:
        return 0  # Not enough proper nouns to compute similarity

    vectors = get_word_vectors(proper_nouns)
    if len(vectors) < 2:
        return 0  # Not enough valid vectors to compute similarity

    # Calculate pairwise cosine similarity
    similarity_matrix = cosine_similarity(vectors)

    # Get the upper triangle of the cosine similarity matrix (excluding diagonal)
    num_pairs = 0
    total_similarity = 0
    for i in range(len(similarity_matrix)):
        for j in range(i+1, len(similarity_matrix)):
            total_similarity += similarity_matrix[i][j]
            num_pairs += 1

    # Return the average cosine similarity
    return total_similarity / num_pairs if num_pairs > 0 else 0



In [ ]:
import numpy as np
df['acs_pn']=np.nan

In [ ]:
df['acs_pn']=df['proper nouns'].apply(average_cosine_similarity)

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

# Download NLTK stopwords if not already downloaded
nltk.download('stopwords')
nltk.download('punkt')

# Step 1: Preprocess text
def preprocess_text_1(text):
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text.lower())  # Tokenize and convert to lowercase
    tokens = [word for word in tokens if word.isalnum() and word not in stop_words]  # Remove stopwords and non-alphanumeric
    return " ".join(tokens)

df['cleaned_text'] = df['texty'].apply(preprocess_text_1)

# Step 2: Convert text to document-term matrix
vectorizer = CountVectorizer(max_df=0.9, min_df=0.2, stop_words='english')
dtm = vectorizer.fit_transform(df['cleaned_text'])

# Step 3: Train LDA model
n_topics = 20  # Number of topics
lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
lda.fit(dtm)

# Step 4: Extract dominant topic for each document
def get_dominant_topic(lda_model, doc_topic_matrix):
    return doc_topic_matrix.argmax(axis=1)

doc_topic_matrix = lda.transform(dtm)
df['dominant_topic'] = get_dominant_topic(lda, doc_topic_matrix)

# Optional: Extract topic keywords
def get_topic_keywords(lda_model, vectorizer, n_top_words=10):
    keywords = []
    for topic_idx, topic in enumerate(lda_model.components_):
        top_words = [vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
        keywords.append(", ".join(top_words))
    return keywords

topic_keywords = get_topic_keywords(lda, vectorizer)

# Step 5: Map topic keywords to dominant topic
df['topic_keywords'] = df['dominant_topic'].apply(lambda x: topic_keywords[x])

# Only keep the 'topic_keywords' column
df_final = df[['topic_keywords']]

# Display the final DataFrame with only topic_keywords
print(df_final)

In [ ]:
df['acs_tk']=np.nan

In [ ]:
df['acs_tk']=df['topic_keywords'].apply(average_cosine_similarity)

In [ ]:
!pip install transformers accelerate


In [ ]:
!pip install transformers
!pip install torch


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch


In [ ]:
# Load SciBERT pre-trained model and tokenizer
model_name = "allenai/scibert_scivocab_uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)


In [ ]:
from transformers import BertTokenizer, BertModel
import torch

# Load pre-trained SciBERT model and tokenizer
model_name = 'allenai/scibert_scivocab_uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# Function to extract embeddings from SciBERT
def get_embeddings(texts, model, tokenizer):
    embeddings = []
    for text in texts:
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
        # Extract the embeddings from the [CLS] token's representation
        embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
        embeddings.append(embedding)
    return embeddings

# Extract embeddings for all texts in the dataset
embeddings = get_embeddings(df['texty'], model, tokenizer)


In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np

In [ ]:
X_train= np.array(embeddings)  # Feature matrix (embeddings)
y_train =df['target'] # Labels (publishable or not)
X_train = X_train.astype(np.float32)  # Convert to float32
y_train = np.array(y_train).astype(np.float32)

# Build a more dense neural network model
model_nn = Sequential()

# Input layer + first hidden layer (increased number of neurons)
model_nn.add(Dense(512, input_dim=X_train.shape[1], activation='relu'))  # Increased number of neurons

# Adding more hidden layers with more neurons to increase model complexity
model_nn.add(Dense(512, activation='relu'))  # Second hidden layer with more neurons
model_nn.add(Dropout(0.5))  # Dropout layer to prevent overfitting

model_nn.add(Dense(256, activation='relu'))  # Third hidden layer with more neurons
model_nn.add(Dense(256, activation='relu'))  # Fourth hidden layer with more neurons
model_nn.add(Dropout(0.5))  # Another Dropout layer

model_nn.add(Dense(128, activation='relu'))  # Fifth hidden layer with more neurons
model_nn.add(Dense(128, activation='relu'))  # Sixth hidden layer with more neurons
model_nn.add(Dropout(0.5))  # Dropout layer

model_nn.add(Dense(64, activation='relu'))  # Seventh hidden layer with more neurons
model_nn.add(Dense(64, activation='relu'))  # Eighth hidden layer with more neurons

# Output layer for binary classification
model_nn.add(Dense(1, activation='sigmoid'))

# Compile the model
model_nn.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model_nn.fit(X_train, y_train, epochs=25, batch_size=4, verbose=1)

# Evaluate the model
loss, accuracy = model_nn.evaluate(X_train, y_train)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

In [ ]:
Q=np.array(embeddings)
Q = Q.astype(np.float32)  # Convert to float32

y_pred = model_nn.predict(Q)


In [ ]:
for i in range(len(y_pred)):
  df['target_nn'][i] = y_pred[i]

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

# Assuming the DataFrame is named df
# Selecting input features and target variable
X = df[['acs_tk', 'acs_pn', 'cluster', 'target_nn']]
y = df['target']

# Initialize the Decision Tree Regressor
regressor = DecisionTreeRegressor(random_state=42)

# Train the model
regressor.fit(X, y)

# Make predictions on the test set
y_fin = regressor.predict(X_test)


In [ ]:
df['target_fin']=y_fin

#TASK 2

In [ ]:
#Done by SciBert Model
gg=[
    "cvpr", "neurips", "cvpr", "neurips", "emnlp", "neurips", "neurips", "kdd", "kdd",
    "neurips", "emnlp", "cvpr", "cvpr", "neurips", "cvpr", "neurips", "kdd", "neurips",
    "cvpr", "cvpr", "cvpr", "neurips", "cvpr", "neurips", "emnlp", "emnlp", "enmnlp",
    "neurips", "emnlp", "emnlp", "cvpr", "neurips", "cvpr", "cvpr", "neurips", "neurips",
    "neurips", "kdd", "emnlp", "kdd", "neurips", "cvpr", "neurips", "neurips", "neurips",
    "neurips", "neuips", "emnlp", "emnlp", "neurips", "neurips", "cvpr", "neurips", "neurips",
    "neurips", "neurips", "neurips", "neurips", "neurips", "neurips", "neurips", "kdd", "cvpr",
    "neurips", "cvpr", "neurips", "neurips", "kdd", "tmlr", "neurips", "neurips", "emnlp",
    "emnlp", "neurips", "kdd", "neurips", "neurips", "neurips", "kdd", "cvpr", "emnlp", "emnlp",
    "neurips", "kdd", "emnlp", "kdd", "emnlp", "neurips", "neurips", "emnlp", "cvpr", "neurips",
    "tlmr"
]
df['conf']=np.nan
j=0
for i in range(135):
  if df['target'][i+15]==1:
    df['conf'][i+15]=gg[j]
    j=j+1
  else:
    df['conf'][i+15]="na"


NameError: name 'df' is not defined

In [ ]:
cvpr_words = [
    "computer vision", "image processing", "object detection", "semantic segmentation", "deep learning",
    "convolutional neural networks", "CNN", "image classification", "feature extraction", "motion analysis",
    "3D reconstruction", "augmented reality", "video analysis", "visual tracking", "face recognition",
    "pose estimation", "generative adversarial networks", "GAN", "data augmentation", "neural networks",
    "object localization", "image captioning", "instance segmentation", "edge detection", "image-to-image",
    "visual localization", "optical flow", "supervised learning", "unsupervised learning", "generative models",
    "attention mechanism", "pose estimation", "graph neural networks", "action recognition", "multi-view learning",
    "object recognition", "unsupervised learning", "face detection", "semantic parsing", "image synthesis",
    "video captioning", "depth estimation", "image segmentation", "multi-modal learning", "scene understanding",
    "image restoration", "face detection", "spatial transformer networks", "image super-resolution",
    "spatial alignment", "image denoising", "neural style transfer", "domain adaptation", "vision transformer",
    "adversarial training", "vision-language models", "attention networks", "multi-object tracking",
    "camera calibration", "object recognition", "region proposals", "image matching", "single-image depth",
    "pose graph optimization", "multi-label classification", "deep reinforcement learning", "optical flow estimation",
    "image filtering", "deep neural networks", "image clustering", "object segmentation", "image captioning",
    "local feature extraction", "neural networks for vision", "model compression", "image recognition",
    "self-supervised learning", "domain adaptation", "image retrieval", "scene parsing", "image labeling",
    "generative image modeling", "differentiable rendering", "loss functions", "non-parametric models",
    "3D object detection", "structure from motion", "scene flow", "3D object tracking", "3D pose estimation",
    "camera pose", "multi-sensor fusion", "multi-task learning", "multi-modal fusion", "image matching algorithms",
    "active learning", "object tracking", "motion capture", "multiview stereo", "computer vision algorithms",
    "3D mesh reconstruction", "point clouds", "object alignment", "neural architecture search", "hyperparameter tuning",
    "feature pooling", "pretrained models", "face landmark detection", "3D face reconstruction", "large-scale datasets",
    "Robustness to noise", "domain generalization", "domain shift", "cross-domain learning", "scene generation",
    "detection frameworks", "human pose tracking", "action classification", "attention models", "semantic labeling",
    "image normalization", "image processing pipelines", "synthetic datasets", "pose-invariant representation",
    "optical flow networks", "pose estimation with deep learning", "unsupervised domain adaptation",
    "natural language processing for vision", "AI-driven image analysis", "neural style transfer algorithms",
    "deep learning for object recognition", "deformable convolution", "image matching networks", "homography estimation",
    "deep learning-based segmentation", "image-to-image translation", "generative models for images",
    "multimodal neural networks", "image captioning and understanding", "learning visual features", "image noise reduction",
    "inpainting models", "deep clustering", "self-supervised models", "contrastive learning", "outlier detection",
    "semantic segmentation architectures", "robust vision systems", "visual transformer networks", "attention-based networks",
    "image classifiers", "object tracking with deep learning", "intra-object learning", "deep learning for medical images",
    "semantic scene parsing", "video object segmentation", "weakly-supervised learning", "image processing with CNNs",
    "image pre-processing", "image-to-text networks", "generative models for scene generation", "stylized image generation",
    "3D shape reconstruction", "interactive segmentation", "interactive object tracking", "feature matching",
    "interactive 3D reconstructions", "neural image compression", "deep generative models", "labeled image segmentation",
    "instance-based learning", "multimodal scene understanding", "real-time video analysis", "motion tracking",
    "video frame prediction", "neural network architectures", "object discovery", "superpixel segmentation",
    "boundary detection", "classification and regression", "nonlinear image mapping", "real-time object recognition",
    "adaptive learning systems", "robotic vision", "video analysis using CNNs", "camera calibration", "real-time tracking",
    "image pre-processing techniques", "deep learning in robotics", "continuous object tracking", "latent space exploration",
    "feature engineering", "detection precision", "scalable image recognition", "multi-scale image processing",
    "image dehazing", "structure-from-motion pipelines", "multi-agent visual recognition", "data synthesis in vision",
    "image segmentation in real-time", "scene reconstruction", "robust 3D reconstruction", "vision systems",
    "transfer learning in vision", "structured learning", "unsupervised depth estimation", "spatial representations",
    "event detection in videos", "3D object recognition", "image matching for medical purposes", "motion segmentation",
    "fuzzy segmentation", "human-object interaction detection", "multi-object classification", "image noise estimation",
    "motion-based image segmentation", "video object detection", "deep learning in autonomous systems", "temporal segmentation",
    "spatial localization", "image-to-image neural networks", "autoencoders", "deep learning for autonomous vehicles",
    "event-based cameras", "multi-object detection in video", "CNN for feature extraction", "supervised vs unsupervised vision",
    "depth from stereo images", "differentiable computer vision models", "multi-frame tracking", "object category detection",
    "crowd counting", "person re-identification", "3D object localization", "sparse image representations",
    "latent variable models", "adaptive neural networks", "image matching algorithms", "optical flow methods",
    "deep neural architectures", "real-time image classification", "recurrent neural networks for vision",
    "generative networks for motion capture", "semantic image restoration", "non-rigid object tracking",
    "deep learning for object localization", "multi-label object recognition", "image segmentation with deep learning",
    "AI-driven vision systems", "feature point matching", "camera network calibration", "training data generation",
    "video segmentation with neural networks", "motion detection algorithms", "object boundary detection", "neural image processing"
]

neurips_words = [
    "reinforcement learning", "machine learning", "deep learning", "generative models", "optimization",
    "neural networks", "probabilistic models", "Bayesian methods", "Markov chains", "support vector machines",
    "unsupervised learning", "supervised learning", "multi-agent systems", "game theory", "transfer learning",
    "meta learning", "neural architecture search", "natural language processing", "representation learning",
    "contextual bandits", "policy gradients", "exploration", "exploitability", "hierarchical reinforcement learning",
    "policy optimization", "actor-critic methods", "attention networks", "neural differential equations",
    "adversarial learning", "Bayesian neural networks", "graph neural networks", "sequence learning", "deep Q-learning",
    "policy iteration", "trust region optimization", "zero-shot learning", "variational inference", "Monte Carlo methods",
    "unsupervised representation learning", "causal inference", "neural differential equations", "deep reinforcement learning",
    "Q-learning", "Bandit problems", "multi-task learning", "long short-term memory", "transformers",
    "Bayesian deep learning", "explainable AI", "meta-learning algorithms", "online learning", "self-supervised learning",
    "unsupervised domain adaptation", "autoencoders", "structured prediction", "stochastic optimization",
    "contrastive learning", "Bayesian optimization", "unsupervised generative models", "transfer reinforcement learning",
    "deep belief networks", "sparse coding", "variational autoencoders", "evolutionary algorithms", "decision trees",
    "data-efficient learning", "policy networks", "DeepMind", "convolutional networks", "continuous control",
    "labeled vs unlabeled learning", "actor-critic reinforcement learning", "opponent modeling", "dynamic programming",
    "policy gradient methods", "deep learning for decision-making", "intrinsic motivation", "representation learning in RL",
    "neural networks for NLP", "language modeling", "structured output prediction", "unsupervised generative adversarial networks",
    "hierarchical models", "convolutional neural networks for RL", "stochastic processes", "real-world RL",
    "unsupervised reinforcement learning", "dual optimization", "deep unsupervised learning", "contrastive divergence",
    "feature selection", "reward shaping", "unsupervised meta-learning", "latent variable models", "backpropagation",
    "random forests", "recurrent neural networks", "attention mechanisms", "convolutional attention networks",
    "deep neural architectures", "Markov decision processes", "multi-agent reinforcement learning", "reinforcement learning algorithms",
    "neural language models", "hierarchical Bayesian models", "deep reinforcement learning for robotics",
    "online optimization", "global optimization", "information theory", "gradient descent methods", "reinforcement learning with limited data",
    "nonlinear optimization", "long-term dependencies", "embedding models", "recurrent attention networks", "knowledge distillation",
    "policy gradient algorithms", "deep hierarchical reinforcement learning", "goal-directed exploration", "self-play",
    "meta-reinforcement learning", "prototypical networks", "deep learning for planning", "deep models for image classification",
    "meta-gradient descent", "adversarial training in RL", "black-box optimization", "statistical learning",
    "transfer learning with deep neural networks", "game-theoretic models", "actor-critic frameworks", "distributional reinforcement learning",
    "graph-based deep learning", "probabilistic graphical models", "model-free reinforcement learning", "optimistic planning",
    "deep feature learning", "long short-term memory networks", "policy exploration", "information bottleneck",
    "sparse reinforcement learning", "deep probabilistic models", "inverse reinforcement learning", "action-value methods",
    "generative adversarial imitation learning", "self-supervised reinforcement learning", "unsupervised policy learning",
    "model-based reinforcement learning", "NLP and machine learning", "contrastive self-supervised learning",
    "backpropagation through time", "differentiable programming", "meta-reasoning", "learning to optimize",
    "attention mechanisms for RL", "interpretable machine learning", "exploration-exploitation trade-off", "long-range dependencies",
    "task-oriented dialogue systems", "data augmentation in reinforcement learning", "active learning", "unsupervised clustering",
    "perceptual loss", "generalization in reinforcement learning", "plan-based reinforcement learning", "policy distillation",
    "affordance learning", "continuous space optimization", "modeling uncertainty", "deep Q-networks", "model-free RL",
    "multi-fidelity optimization", "inverse dynamics", "task-specific reinforcement learning", "unsupervised learning with generative models",
    "exploiting bias in RL", "deep probabilistic programming", "active exploration", "importance sampling", "policy exploration vs exploitation",
    "non-stationary environments", "biological neural networks", "path planning", "reinforcement learning in multi-agent settings",
    "graph-based reinforcement learning", "ensemble learning in RL", "sparse attention networks", "deep multitask learning",
    "policy reuse", "reward prediction", "unsupervised imitation learning", "symbolic learning", "multi-agent exploration",
    "deep generative modeling", "joint learning models", "learning to learn", "meta-reinforcement learning algorithms",
    "contextual reinforcement learning", "adaptive algorithms", "Bayesian deep networks", "reward maximization", "nonlinear reinforcement learning",
    "graph convolutional networks", "policy search", "reinforcement learning for autonomous vehicles", "value iteration networks",
    "neural architecture search for reinforcement learning", "multi-agent coordination", "perceptual reinforcement learning",
    "robotic learning from demonstrations", "fairness in machine learning", "regret minimization", "policy bottleneck",
    "reinforcement learning with safety constraints", "stochastic policies", "non-convex optimization", "deep learning in robotics",
    "meta-learning for RL", "trust region methods", "multi-arm bandit", "zero-shot RL", "graph models for RL",
    "curriculum learning", "neural networks for robotics", "adversarial imitation learning", "environment modeling in RL",
    "learning-to-rank", "few-shot learning", "active adversarial learning", "data-driven optimization", "exploration-driven learning",
    "ensemble policy methods", "policy learning with limited supervision", "online reinforcement learning",
    "interventional reinforcement learning", "neural networks for generative modeling", "meta-gradient optimization",
    "generative reinforcement learning", "optimization in deep learning", "sparse Q-learning", "exploration with adversarial training",
    "visual reinforcement learning", "spatial attention", "decision-making models", "task decomposition", "end-to-end reinforcement learning",
    "reinforcement learning for multi-agent systems", "meta-policy optimization", "large-scale reinforcement learning", "dual-Q learning",
    "spatiotemporal reinforcement learning", "transferable policies", "unsupervised predictive learning", "action generation",
    "imitation learning with adversarial rewards", "reward engineering", "multi-objective reinforcement learning",
    "information-theoretic optimization", "recurrent neural reinforcement learning", "task-oriented reinforcement learning"
]
emnlp_words = [
    "natural language processing", "NLP", "sentiment analysis", "text classification", "tokenization",
    "part-of-speech tagging", "named entity recognition", "semantic parsing", "question answering", "machine translation",
    "language models", "transformers", "BERT", "GPT", "word embeddings", "word2vec", "GloVe", "dependency parsing",
    "dialog systems", "sequence-to-sequence", "positional encoding", "neural networks", "transformer networks",
    "attention mechanism", "text summarization", "multi-task learning", "text generation", "sentiment prediction",
    "document classification", "knowledge graphs", "neural machine translation", "text mining", "language generation",
    "word sense disambiguation", "coreference resolution", "syntax trees", "semantic role labeling", "language modeling",
    "neural embeddings", "cross-lingual models", "text mining", "language transfer", "question answering systems",
    "unstructured data", "information retrieval", "question answering", "zero-shot learning", "active learning",
    "fine-tuning", "question answering dataset", "pretrained models", "language adaptation", "speech recognition",
    "language similarity", "semantic similarity", "dialog modeling", "morphological analysis", "token-level attention",
    "sequence labeling", "dependency trees", "semantic parsing", "neural text classification", "unsupervised learning",
    "text entailment", "universal sentence encoder", "extractive summarization", "abstractive summarization",
    "data augmentation", "multilingual NLP", "deep learning", "autoencoders", "word alignment", "speech-to-text",
    "text-to-speech", "text-to-text", "document clustering", "word clustering", "sentence embeddings", "parsing strategies",
    "constituency parsing", "sentiment classification", "word frequency", "frequency-based models", "deep learning for NLP",
    "multi-lingual word embeddings", "stochastic gradient descent", "text processing pipelines", "dialog act classification",
    "semantic similarity modeling", "predictive modeling", "language model pretraining", "token classification",
    "zero-shot text classification", "information extraction", "entity linking", "relation extraction", "cross-domain learning",
    "chatbots", "dialog systems", "language generation", "question answering evaluation", "transfer learning for NLP",
    "neural network-based text generation", "unsupervised translation", "sentence generation", "language understanding",
    "text-based reasoning", "automatic summarization", "sparse attention", "text alignment", "deep neural networks for NLP",
    "pretrained language models", "multi-label classification", "text vectorization", "lexical semantics", "dependency grammar",
    "morphological modeling", "linguistic annotations", "neural sequence models", "transformers for sequence-to-sequence",
    "masking in transformers", "word embeddings for classification", "token-level embeddings", "hidden states in NLP",
    "language modeling with transformers", "bidirectional transformers", "attention-based models", "parse tree construction",
    "machine translation evaluation", "language generation models", "pretraining language models", "text simplification",
    "intent classification", "semantic segmentation", "syntactic parsing", "cross-lingual embeddings", "sentence parsing",
    "pretrained transformers", "contextual word embeddings", "contextualized representations", "error analysis in NLP",
    "linear chain CRFs", "perceptron-based models", "machine translation evaluation metrics", "sentiment analysis using LSTMs",
    "active learning for NLP", "question answering evaluation metrics", "dialog management", "character-level models",
    "attention-based sequence models", "subword units", "sentiment analysis in dialogues", "co-reference resolution algorithms",
    "conditional random fields", "deep reinforcement learning for NLP", "social media text processing", "neural network classifiers",
    "textual entailment", "aspect-based sentiment analysis", "pragmatics in NLP", "text classification benchmarks",
    "syntactic trees", "lexical analysis", "named entity recognition models", "computational linguistics", "pos tagging models",
    "machine reading comprehension", "NLP for information retrieval", "sentence classification", "deep learning in machine translation",
    "dual attention networks", "speech recognition accuracy", "word vectors for NLP", "dialog dataset", "contextual embedding",
    "conversational agents", "machine translation corpus", "syntactic accuracy", "knowledge extraction", "language-based features",
    "neural sequence-to-sequence learning", "inter-sentence relations", "language model fine-tuning", "transformer attention layers",
    "multimodal NLP", "word-level attention", "subword tokenization", "word tokenization methods", "language model transfer learning",
    "dependency parsing algorithms", "coreference resolution methods", "unsupervised sequence labeling", "generation-based NLP tasks",
    "language resource construction", "text normalization", "morphology-aware neural networks", "task-specific embeddings",
    "language generation from scratch", "language understanding in conversational agents", "unsupervised clustering of texts",
    "sequence-to-sequence prediction", "domain adaptation in NLP", "sequence generation in NLP", "cross-lingual sentiment analysis",
    "document-level sentiment analysis", "automatic text generation", "embedding-based models for NLP", "sequence classification",
    "long short-term memory networks", "latent variable models for NLP", "document parsing", "lexicon-based text classification",
    "online learning in NLP", "multilingual BERT", "continuous bag-of-words", "skip-gram model", "semantic parsing in dialogues",
    "multi-turn dialog", "text classification algorithms", "pretrained language model fine-tuning", "token-level text classification",
    "evaluation metrics for NLP tasks", "long document classification", "labeled data generation", "zero-shot learning for language tasks",
    "neural word embeddings", "multi-modal embeddings", "text retrieval", "fine-grained sentiment analysis", "speech-to-text transcription",
    "question generation", "question answering with transformers", "language modeling with transformers", "semantic role labeling",
    "unsupervised learning for text", "preprocessing text data", "text vectorization strategies", "meta-learning for NLP",
    "fact-checking", "contextualized word representations", "dialog act recognition", "dialog management systems", "polarity detection",
    "semantic anomaly detection", "named entity disambiguation", "cross-domain knowledge transfer", "word clustering",
    "complex question answering", "deep learning for syntactic parsing", "word parsing models", "textual entailment models",
    "transformers for text classification", "graph-based sentence classification", "lexical chaining", "unsupervised learning in NLP",
    "textual data augmentation", "syntax-based text classification", "data-efficient machine learning", "document-level NLP",
    "neural sequence models for NLP", "machine translation with transformers", "rule-based systems", "topic modeling",
    "sentence entailment", "attention-based text classification", "pretrained word embeddings", "language processing with LSTMs",
    "co-reference modeling", "dialog evaluation metrics", "inference modeling for NLP", "speech act theory in NLP", "vocabulary expansion",
    "deep learning for text mining", "unsupervised translation", "neural network-based question answering", "semantic vector space models",
    "fuzzy logic for NLP", "domain adaptation models", "hybrid NLP models", "task-specific neural networks", "in-context learning",
    "syntactic ambiguity resolution", "contrastive learning in NLP", "sequence labeling models", "multi-task learning for NLP",
    "latent semantic analysis", "topic modeling algorithms", "embedding similarity", "syntactic analysis for NLP",
    "transformer-based models for NLP", "multilingual models", "neural classification models", "discriminative training models",
    "contextual embeddings in NLP", "unsupervised neural network learning", "long-term dependencies in NLP", "question answering corpus",
    "text normalization techniques", "natural language inference", "token prediction", "document representation models",
    "fine-grained classification", "data-driven language models", "aspect-based classification", "semantic parsing algorithms"
]


tmlr_words = [
    "machine learning", "theory of machine learning", "algorithmic complexity", "online learning", "optimization",
    "convex optimization", "reinforcement learning", "learning theory", "statistical learning", "generalization",
    "sample complexity", "no-regret algorithms", "online algorithms", "non-convex optimization", "stochastic processes",
    "learning with noise", "learning dynamics", "information theory", "hypothesis testing", "PAC learning",
    "gradient descent", "nonparametric methods", "learning algorithms", "kernel methods", "bandit problems",
    "robust learning", "empirical risk minimization", "complexity bounds", "game theory", "approximation algorithms",
    "multi-agent learning", "multi-armed bandits", "regularization", "statistical inference", "approximate inference",
    "cross-validation", "feature selection", "adversarial robustness", "strongly convex optimization", "dual optimization",
    "data-driven optimization", "adaptive learning rates", "Bayesian optimization", "convergence analysis", "decision theory",
    "stochastic gradient descent", "Markov decision processes", "deep reinforcement learning", "duality theory", "convexity",
    "Lipschitz continuity", "backpropagation", "complexity theory", "information gain", "maximum likelihood estimation",
    "empirical risk minimization", "principle of maximum entropy", "Fisher information", "expectation-maximization algorithm",
    "gradient flow", "Lagrangian relaxation", "robust optimization", "expected loss minimization", "approximation error",
    "Bregman divergence", "training algorithms", "probabilistic graphical models", "belief propagation", "Bayesian networks",
    "quadratic programming", "combinatorial optimization", "multi-task learning", "generative models", "regularized regression",
    "hierarchical models", "convex hull", "empirical distribution", "gradient boosting", "stochastic approximation",
    "maximum entropy", "information theory", "adversarial training", "ReLU activation function", "variational inference",
    "spatial attention", "adaptive regularization", "probabilistic modeling", "Markov chains", "metropolis-Hastings algorithm",
    "latent variable models", "bounded rationality", "second-order methods", "low-rank approximation", "multi-class classification",
    "ensemble methods", "bootstrap aggregation", "information-theoretic bounds", "KL divergence", "deep neural networks",
    "neural architecture search", "optimization landscape", "decision trees", "conditional entropy", "training loss functions",
    "finite-sample analysis", "gradient-based methods", "random forests", "neural Turing machines", "graphical models",
    "convex relaxations", "primal-dual methods", "support vector machines", "structured prediction", "backpropagation algorithms",
    "forward-backward algorithm", "second-order optimization", "quadratic optimization", "active learning", "convergence rate",
    "nonlinear optimization", "sample size analysis", "MCMC methods", "meta-learning", "combinatorial game theory",
    "discrete optimization", "primal-dual optimization", "zero-shot learning", "approximate Bayesian computation", "sparse optimization",
    "gradient tracking", "dynamic programming", "policy optimization", "adaptive sampling", "multi-objective optimization",
    "policy gradient methods", "stochastic approximation", "regret analysis", "algorithmic fairness", "noisy gradient methods",
    "Monte Carlo methods", "stochastic games", "model selection", "information-theoretic learning", "joint distribution modeling",
    "parameter estimation", "empirical process theory", "controlled Markov chains", "expected value minimization",
    "constraint satisfaction problems", "high-dimensional statistics", "combinatorial optimization problems", "subgradient methods",
    "probabilistic programming", "sparsity-promoting methods", "exploration-exploitation tradeoff", "causal inference",
    "zero-sum games", "nash equilibrium", "utility maximization", "robust regression", "submodular optimization",
    "Gaussian processes", "Bayesian statistics", "minimax optimization", "exploration algorithms", "conditional probability",
    "combinatorial auctions", "sequential decision making", "generalization error", "empirical risk minimization",
    "online convex optimization", "stochastic gradient methods", "local search algorithms", "gradient-based optimization",
    "saddle point problems", "structured sparsity", "information geometry", "linear classifiers", "subspace learning",
    "ensemble learning", "quadratic functions", "learning with constraints", "feature engineering", "model complexity",
    "randomized algorithms", "dimension reduction", "quadratic forms", "minimum variance analysis", "complexity classes",
    "graph optimization", "MCMC sampling", "Bayesian model selection", "decision making under uncertainty",
    "constraint-based optimization", "large margin classifiers", "state space exploration", "POMDPs", "information gain maximization",
    "differential privacy", "SVM kernel tricks", "convex hull algorithms", "approximation ratios", "machine learning algorithms",
    "algorithmic randomness", "structured sparsity constraints", "subspace clustering", "deep Q-learning", "data assimilation",
    "support vector regression", "sparse signal recovery", "fast gradient methods", "convergence rate analysis",
    "Markov decision processes", "generalized linear models", "structural risk minimization", "multivariate statistics",
    "rich-get-richer models", "planar graphs", "fixed-point iteration", "adaptive filtering", "L1 regularization",
    "Bayesian learning", "adaptive filtering", "L1 minimization", "interpolation methods", "maximum a posteriori estimation",
    "approximation of convex functions", "single-agent optimization", "robust game theory", "statistical decision theory",
    "generalization bounds", "direct search algorithms", "global optimization", "constraint optimization", "active set methods",
    "long-term average cost", "graph matching", "game-theoretic optimization", "convex hull algorithms", "stochastic process models",
    "noisy channel models", "hierarchical Bayesian models", "maximum likelihood models", "multi-agent systems", "inverse reinforcement learning",
    "discrete gradient descent", "stochastic gradient methods", "decision rules", "dynamic optimization", "multistage optimization",
    "integer programming", "multi-agent reinforcement learning", "multi-armed bandit problems", "supervised learning",
    "unsupervised learning", "non-linear dynamics", "block coordinate descent", "mean-field games", "maximum likelihood estimation",
    "decision trees", "bandit algorithms", "regression trees", "loss function", "primal-dual optimization", "information-theoretic inequalities",
    "quadratic loss", "high-dimensional models", "empirical error", "conditional entropy", "maximum posterior estimation",
    "sequential learning", "nonlinear dynamics", "Bayesian posterior", "information-theoretic bounds", "asymptotic analysis",
    "generalization capacity", "low-rank matrix approximation", "minimization of convex functions", "feedback loops",
    "sample-efficient algorithms", "rate-distortion theory", "bias-variance tradeoff", "upper bound estimation", "approximation theory",
    "multi-dimensional scaling", "regularized least squares", "support vector regression", "multi-class SVM", "deep belief networks",
    "Gaussian mixture models", "hierarchical clustering", "complexity measures", "computational statistics", "Markov random fields",
    "robust statistics", "MCMC estimation", "reinforcement learning theory", "multi-agent coordination", "Bayesian inference",
    "optimal stopping theory", "multivariate regression", "game theory equilibrium", "non-convex losses", "learning dynamics",
    "neural architecture search", "optimal transport", "low-complexity algorithms", "learning with feedback", "spectral clustering",
    "random matrix theory", "multivariate time series", "high-dimensional data", "subsampling algorithms", "simulated annealing",
    "domain adaptation", "generalization gap", "asymptotic behavior", "reinforcement learning convergence", "information flow",
    "generalized Bregman divergences", "neural dynamic systems", "regret minimization", "stochastic control", "causal modeling",
    "bandit algorithms", "policy gradient methods", "structured prediction tasks", "parameterized algorithms", "graph cuts",
    "entropy regularization", "robust optimization models", "feature engineering techniques", "minimax theory", "universal approximation",
    "quadratic forms", "linear regression", "L1/L2 regularization", "convergence diagnostics", "asymptotic efficiency", "perturbation analysis"
]


kdd_words = [
    "data mining", "knowledge discovery", "big data", "machine learning", "data analytics", "data preprocessing",
    "clustering", "association rule mining", "pattern recognition", "graph mining", "social network analysis",
    "anomaly detection", "classification", "supervised learning", "unsupervised learning", "time series analysis",
    "recommendation systems", "neural networks", "ensemble methods", "feature extraction", "outlier detection",
    "multi-view learning", "spatial data mining", "deep learning", "reinforcement learning", "data fusion", "data pipelines",
    "data privacy", "model evaluation", "predictive analytics", "sentiment analysis", "market basket analysis",
    "classification algorithms", "decision trees", "random forests", "support vector machines", "graph theory",
    "predictive modeling", "predictive systems", "outlier detection", "deep reinforcement learning", "data visualization",
    "complex networks", "graph theory", "spatial analytics", "distributed systems", "cloud computing", "model deployment",
    "data integration", "data transformation", "data wrangling", "data cleaning", "data augmentation", "feature engineering",
    "feature selection", "dimensionality reduction", "principal component analysis", "PCA", "t-SNE", "k-means", "k-nearest neighbors",
    "SVD", "singular value decomposition", "matrix factorization", "latent variable models", "hierarchical clustering",
    "DBSCAN", "Gaussian mixture models", "hidden Markov models", "autoencoders", "word embeddings", "semantic analysis",
    "text mining", "text classification", "topic modeling", "latent Dirichlet allocation", "LDA", "bag of words",
    "TF-IDF", "NLP", "tokenization", "stopword removal", "word2vec", "fastText", "glove", "sentence embeddings", "BERT",
    "transformers", "transfer learning", "cross-validation", "holdout validation", "model selection", "hyperparameter tuning",
    "hyperparameter optimization", "grid search", "random search", "Bayesian optimization", "cross-validation folds",
    "validation sets", "train-test split", "data splitting", "ROC curve", "precision", "recall", "F1 score", "AUC",
    "confusion matrix", "classification report", "error rate", "sensitivity", "specificity", "outlier analysis", "feature maps",
    "convolutional neural networks", "CNN", "fully connected networks", "deep neural networks", "multi-layer perceptrons",
    "backpropagation", "gradient descent", "stochastic gradient descent", "mini-batch gradient descent", "ADAM optimizer",
    "learning rate", "batch normalization", "dropout", "activation functions", "sigmoid", "tanh", "ReLU", "leaky ReLU",
    "softmax", "BCE loss", "categorical cross-entropy", "mean squared error", "mean absolute error", "loss functions",
    "regularization", "L1 regularization", "L2 regularization", "ridge regression", "lasso regression", "elastic net",
    "support vector regression", "kernel methods", "Gaussian kernels", "polynomial kernels", "RBF kernels", "kernel trick",
    "principal component regression", "linear regression", "logistic regression", "ridge regression", "adaptive boosting",
    "gradient boosting", "XGBoost", "LightGBM", "CatBoost", "AdaBoost", "bagging", "stacking", "voting classifiers",
    "machine learning pipelines", "model training", "model testing", "model deployment", "CI/CD for ML", "model monitoring",
    "model drift", "data drift", "model retraining", "real-time analytics", "streaming data", "data warehouses",
    "data lakes", "big data processing", "Apache Hadoop", "Apache Spark", "MapReduce", "distributed computing",
    "distributed learning", "cloud-based data storage", "cloud-based data processing", "Kubernetes", "Docker", "ML Ops",
    "continuous integration", "continuous deployment", "API integration", "batch processing", "real-time processing",
    "data science workflows", "Jupyter notebooks", "Google Colab", "machine learning APIs", "model endpoints", "data models",
    "relational databases", "NoSQL", "MongoDB", "Cassandra", "HBase", "PostgreSQL", "SQL databases", "data retrieval",
    "data cleaning algorithms", "data imputation", "missing value imputation", "data normalization", "data standardization",
    "data scaling", "data transformations", "standard scaling", "min-max scaling", "robust scaling", "feature extraction",
    "audio analysis", "speech recognition", "image recognition", "object detection", "computer vision", "semantic segmentation",
    "instance segmentation", "YOLO", "Fast R-CNN", "Mask R-CNN", "ResNet", "VGG", "CNN architectures", "image processing",
    "image augmentation", "edge detection", "Hough transform", "convolution layers", "pooling layers", "fully connected layers",
    "transfer learning models", "pre-trained models", "model fine-tuning", "hyperparameter search", "ensemble learning",
    "meta-learning", "online learning", "active learning", "semi-supervised learning", "unsupervised learning techniques",
    "multi-class classification", "multi-label classification", "imbalanced datasets", "class imbalance", "SMOTE",
    "oversampling", "undersampling", "cost-sensitive learning", "fairness in machine learning", "explainability in ML",
    "model interpretability", "SHAP values", "LIME", "counterfactual explanations", "trustworthy AI", "bias in ML",
    "algorithmic fairness", "privacy-preserving machine learning", "Federated learning", "differential privacy",
    "secure machine learning", "encrypted data analysis", "synthetic data generation", "data anonymization",
    "data leakage", "feature importance", "importance sampling", "importance-weighted classification", "data-driven decision making",
    "risk assessment", "predictive maintenance", "fraud detection", "insurance modeling", "healthcare analytics",
    "customer churn prediction", "market segmentation", "sales forecasting", "stock market prediction", "weather forecasting",
    "smart cities", "IoT data analysis", "sensor data analysis", "biomedical data analysis", "genomic data analysis",
    "clinical trials analysis", "telemedicine data", "sports analytics", "e-commerce analytics", "consumer behavior analysis",
    "recommendation algorithms", "collaborative filtering", "content-based filtering", "matrix factorization", "latent factors",
    "user-item interaction", "cold start problem", "click-through rate prediction", "A/B testing", "user experience analytics",
    "sentiment classification", "emotion detection", "text classification", "document clustering", "entity recognition",
    "named entity recognition", "topic clustering", "web mining", "social media mining", "web scraping", "data crawling",
    "data annotation", "data labeling", "labeling tools", "crowdsourcing for data labeling", "data validation", "data governance",
    "data ethics", "GDPR compliance", "data stewardship", "data provenance", "data lineage", "data quality", "data accuracy",
    "data consistency", "data completeness", "data validity", "data redundancy", "data consistency checks", "data traceability",
    "data warehousing", "ETL processes", "data pipeline automation", "batch data processing", "data synchronization",
    "event-driven data processing", "data shuffling", "feature pipeline", "data pipeline architecture", "data processing frameworks",
    "IoT analytics", "real-time dashboards", "business intelligence", "executive analytics", "enterprise data analysis",
    "smart analytics", "data democratization", "AI-driven data science", "data visualization tools", "Power BI", "Tableau",
    "Matplotlib", "Seaborn", "Plotly", "D3.js", "data storytelling", "data-driven insights", "interactive visualizations",
    "geospatial data analysis", "location-based services", "GPS data analysis", "map-based data visualization", "geospatial prediction",
    "geo-fencing", "spatial data models", "spatio-temporal data", "geospatial clustering", "geospatial pattern recognition",
    "geo-statistics", "multivariate analysis", "cognitive computing", "data-centric AI", "data pipeline management",
    "data system monitoring", "elastic search", "big data analytics", "real-time data stream processing", "data warehouse architecture",
    "data lakehouse", "data mesh architecture", "databases as a service", "cloud data solutions", "hybrid cloud solutions",
    "data orchestration", "distributed analytics", "data mining applications", "big data challenges", "data science operations"
]

In [ ]:
conference_keywords={"CVPR":cvpr_words,"NUERIPS":neurips_words,"EMNLP":emnlp_words,"TMLR":tmlr_words,"KDD":kdd_words}

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Step 1: Compute embeddings for each word in conference keywords
def compute_word_embeddings(keywords, model, tokenizer):
    """
    Compute embeddings for each word in the conference keywords using SciBERT.

    Args:
        keywords (list): List of keywords for the conference.
        model: SciBERT model.
        tokenizer: SciBERT tokenizer.

    Returns:
        list: List of word embeddings for the keywords.
    """
    embeddings = []
    for keyword in keywords:
        inputs = tokenizer(keyword, return_tensors="pt", truncation=True, padding=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
        embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
        embeddings.append(embedding)
    return embeddings


# Compute word embeddings for each conference
conference_word_embeddings = {
    conference: compute_word_embeddings(keywords, model, tokenizer)
    for conference, keywords in conference_keywords.items()
}

# Step 2: Compute average similarity for a paper with each conference
def compute_average_similarity(paper_embedding, conference_word_embeddings):
    """
    Compute the average cosine similarity between a paper's embedding and a conference's word embeddings.

    Args:
        paper_embedding (numpy.array): Embedding of the research paper.
        conference_word_embeddings (list): List of word embeddings for a conference.

    Returns:
        float: The average cosine similarity.
    """
    similarities = []
    for word_embedding in conference_word_embeddings:
        similarity = cosine_similarity([paper_embedding], [word_embedding])[0, 0]
        similarities.append(similarity)
    return np.mean(similarities) if similarities else 0

# Step 3: Assign conference to each paper
def assign_conference_by_similarity(paper_embedding, conference_word_embeddings):
    """
    Assign the most suitable conference to a paper based on average similarity.

    Args:
        paper_embedding (numpy.array): Embedding of the research paper.
        conference_word_embeddings (dict): Dictionary of word embeddings for all conferences.

    Returns:
        str: The name of the most similar conference.
    """
    best_conference = None
    max_similarity = -1

    for conference, word_embeddings in conference_word_embeddings.items():
        avg_similarity = compute_average_similarity(paper_embedding, word_embeddings)
        if avg_similarity > max_similarity:
            max_similarity = avg_similarity
            best_conference = conference

    return best_conference

# Step 4: Predict conferences for all papers
df['predicted_conference'] = [
    assign_conference_by_similarity(embedding, conference_word_embeddings)
    for embedding in embeddings
]

# Output predictions
print(df[['texty', 'predicted_conference']])


In [ ]:
df['predicted_conference'].value_counts()

In [ ]:
from scipy.spatial.distance import euclidean
import numpy as np

# Step 1: Precompute average embeddings for each conference
def compute_average_embedding(keywords, model, tokenizer):
    """
    Compute the average embedding for a list of keywords using SciBERT.

    Args:
        keywords (list): List of keywords for a conference.
        model: SciBERT model.
        tokenizer: SciBERT tokenizer.

    Returns:
        numpy.array: Average embedding of the keywords.
    """
    embeddings = []
    for keyword in keywords:
        inputs = tokenizer(keyword, return_tensors="pt", truncation=True, padding=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
        embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
        embeddings.append(embedding)
    return np.mean(embeddings, axis=0) if embeddings else np.zeros(model.config.hidden_size)

# Precompute average embeddings for all conferences
conference_avg_embeddings = {
    conference: compute_average_embedding(keywords, model, tokenizer)
    for conference, keywords in conference_keywords.items()
}

# Step 2: Compute Euclidean distance and assign conference
def assign_conference_by_euclidean(paper_embedding, conference_avg_embeddings):
    """
    Assign the closest conference to a paper based on Euclidean distance.

    Args:
        paper_embedding (numpy.array): Embedding of the research paper.
        conference_avg_embeddings (dict): Dictionary of average embeddings for all conferences.

    Returns:
        str: The name of the closest conference.
    """
    best_conference = None
    min_distance = float('inf')

    for conference, avg_embedding in conference_avg_embeddings.items():
        distance = euclidean(paper_embedding, avg_embedding)
        if distance < min_distance:
            min_distance = distance
            best_conference = conference

    return best_conference

# Step 3: Predict conferences for all papers
df['predicted_conference_2'] = [
    assign_conference_by_euclidean(embedding, conference_avg_embeddings)
    for embedding in embeddings
]

# Output predictions
print(df[['texty', 'predicted_conference_2']])


In [ ]:
conference_keywords_2 = {
    "CVPR": [
        "computer vision", "image processing", "object detection", "image segmentation", "3D reconstruction",
        "pose estimation", "scene understanding", "visual recognition", "optical flow", "action recognition",
        "video analysis", "visual tracking", "image classification", "stereo vision", "depth estimation",
        "feature extraction", "visual data", "image restoration", "camera calibration", "visual feature matching",
        "image synthesis", "neural networks for vision", "convolutional neural networks", "generative adversarial networks",
        "scene analysis", "motion analysis", "visual perception", "spatial transformations"
    ],
    "EMNLP": [
        "natural language processing", "text mining", "machine translation", "text summarization", "question answering",
        "dialogue systems", "semantic parsing", "named entity recognition", "text generation", "sentiment analysis",
        "language modeling", "text classification", "dependency parsing", "transformers", "word embeddings", "language understanding",
        "syntax parsing", "discourse analysis", "textual data", "tokenization", "speech recognition",
        "morphological analysis", "semantic analysis", "cross-lingual tasks", "language resources",
        "question generation", "coreference resolution", "textual entailment", "neural machine translation"
    ],
    "NEURIPS": [
        "machine learning", "reinforcement learning", "deep learning", "neural networks", "variational inference",
        "self-supervised learning", "meta-learning", "federated learning", "generative models", "optimization",
        "Bayesian inference", "graph neural networks", "unsupervised learning", "supervised learning", "neural architecture",
        "model training", "probabilistic models", "decision making", "active learning", "clustering",
        "evolutionary algorithms", "data-driven models", "causal inference", "statistical models", "optimization algorithms",
        "transformer models", "computational neuroscience", "deep reinforcement learning", "algorithmic fairness"
    ],
    "TMLR": [
        "machine learning theory", "learning theory", "theoretical ML", "generalization bounds", "PAC learning",
        "non-convex optimization", "convex optimization", "kernel methods", "spectral methods", "dimensionality reduction",
        "statistical learning", "algorithmic fairness", "information theory", "mathematical foundations", "optimization theory",
        "function approximation", "bias-variance tradeoff", "learning algorithms", "statistical models", "generalization error",
        "theoretical analysis", "robust learning", "stochastic processes", "mathematical optimization", "empirical risk minimization"
    ],
    "KDD": [
        "data mining", "data analysis", "big data", "knowledge discovery", "data-driven decision making",
        "anomaly detection", "data visualization", "feature engineering", "scalable algorithms", "graph mining",
        "spatiotemporal analysis", "recommendation systems", "predictive modeling", "real-time data processing",
        "unsupervised learning", "classification", "data preprocessing", "clustering", "association rule learning",
        "data pipelines", "pattern recognition", "data integration", "data exploration", "forecasting",
        "decision support systems", "data retrieval", "large-scale data", "data quality", "behavior analysis"
    ]
}


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Step 1: Compute embeddings for each word in conference keywords
def compute_word_embeddings(keywords, model, tokenizer):
    """
    Compute embeddings for each word in the conference keywords using SciBERT.

    Args:
        keywords (list): List of keywords for the conference.
        model: SciBERT model.
        tokenizer: SciBERT tokenizer.

    Returns:
        list: List of word embeddings for the keywords.
    """
    embeddings = []
    for keyword in keywords:
        inputs = tokenizer(keyword, return_tensors="pt", truncation=True, padding=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
        embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
        embeddings.append(embedding)
    return embeddings


# Compute word embeddings for each conference
conference_word_embeddings = {
    conference: compute_word_embeddings(keywords, model, tokenizer)
    for conference, keywords in conference_keywords_2.items()
}

# Step 2: Compute average similarity for a paper with each conference
def compute_average_similarity(paper_embedding, conference_word_embeddings):
    """
    Compute the average cosine similarity between a paper's embedding and a conference's word embeddings.

    Args:
        paper_embedding (numpy.array): Embedding of the research paper.
        conference_word_embeddings (list): List of word embeddings for a conference.

    Returns:
        float: The average cosine similarity.
    """
    similarities = []
    for word_embedding in conference_word_embeddings:
        similarity = cosine_similarity([paper_embedding], [word_embedding])[0, 0]
        similarities.append(similarity)
    return np.mean(similarities) if similarities else 0

# Step 3: Assign conference to each paper
def assign_conference_by_similarity(paper_embedding, conference_word_embeddings):
    """
    Assign the most suitable conference to a paper based on average similarity.

    Args:
        paper_embedding (numpy.array): Embedding of the research paper.
        conference_word_embeddings (dict): Dictionary of word embeddings for all conferences.

    Returns:
        str: The name of the most similar conference.
    """
    best_conference = None
    max_similarity = -1

    for conference, word_embeddings in conference_word_embeddings.items():
        avg_similarity = compute_average_similarity(paper_embedding, word_embeddings)
        if avg_similarity > max_similarity:
            max_similarity = avg_similarity
            best_conference = conference

    return best_conference

# Step 4: Predict conferences for all papers
df['predicted_conference_3'] = [
    assign_conference_by_similarity(embedding, conference_word_embeddings)
    for embedding in embeddings
]

# Output predictions
print(df[['texty', 'predicted_conference_3']])


In [ ]:
df['predicted_conference_3'].value_counts()

In [ ]:
# install dependencies
!pip install transformers
!pip install einops
!pip install accelerate

In [ ]:
!pip install sentencepiece


In [ ]:
import torch                        # allows Tensor computation with strong GPU acceleration
from transformers import pipeline   # fast way to use pre-trained models for inference
import os

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, pipeline
# Load pre-trained T5 model and tokenizer
model_name = 't5-large'  # You can also use 't5-base', 't5-large' based on your needs
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


In [ ]:
df["Abst"]=np.nan
for i in range(150):
  df["Abst"][i]=df['text_dict'][i]["Abstract"]

In [ ]:
from transformers import pipeline

# Initialize the T5 model and tokenizer for text-to-text generation
question_answerer = pipeline("text2text-generation", model="t5-small", tokenizer="t5-small")

# Define a function to generate conference justification based on the paper's abstract
def generate_conference_classification(paper_abstract):
    # List of conferences (this can be expanded or modified as needed)
    conferences = ["kdd", "neurips", "emnlp", "cvpr", "tmlr"]

    # Generate a question about which conference the paper belongs to
    question = f"Which conference does this paper belong to among {', '.join(conferences)}?"

    kk=paper_abstract+textt

    textt="KDD focuses on data mining, NeurIPS on neural networks and AI, EMNLP on natural language processing, CVPR on computer vision, and TMLR on reproducible machine learning research"

    # Combine the question with the context (paper content)
    input_text = f"question: {question} context: {kk}"

    # Check if the input text exceeds the model's token limit (512 tokens for T5-small)
    tokenizer = question_answerer.tokenizer
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True, max_length=512)

    # Generate the answer with the model
    answer = question_answerer(input_text, max_length=512, do_sample=False)

    # Return the generated justification (answer)
    return answer[0]['generated_text']



In [ ]:
df['conf_4']=np.nan
for i in range(150):
  df['conf_4'][i]=generate_conference_classification(df["Abst"][i])


In [ ]:
def prioritize_predictions(pred1, pred2, pred3, pred4, pred5):
    """
    Prioritize predictions based on a fair balance of weights and consensus:
    1. pred1 has the highest priority but can be overridden if a strong consensus exists.
    2. Check for consensus among pred2, pred3, pred4, and pred5.
    3. If no strong consensus, prioritize pred1.
    4. Use a fallback value like "Undecided" if all predictions are None.
    """
    from collections import Counter

    # Count the occurrences of each prediction (ignoring None)
    predictions = [pred2, pred3, pred4, pred5]
    prediction_counts = Counter(p for p in predictions if p is not None)

    # Determine the most common prediction and its frequency
    most_common, frequency = prediction_counts.most_common(1)[0] if prediction_counts else (None, 0)

    # Rule 1: If pred1 is not None and aligns with consensus, prioritize pred1
    if pred1 is not None and pred1 == most_common:
        return pred1

    # Rule 2: If a strong consensus exists (at least 3 out of 4 agree), use it
    if frequency >= 3:
        return most_common

    # Rule 3: If no strong consensus, prioritize pred1 if not None
    if pred1 is not None:
        return pred1

    # Rule 4: Default case (no consensus, pred1 is None)
    return None  # or a fallback value like "Undecided"


In [ ]:
df['confp']=df.apply(lambda row: prioritize_predictions(row['conf'],row['predicted_conference'], row['predicted_conference_2'], row['predicted_conference_3'], row['conf_4']), axis=1)

In [ ]:
df["Abst"]

In [ ]:
from transformers import pipeline
import pandas as pd

# Step 1: Initialize the summarizer pipeline
summarizer = pipeline("summarization", model="t5-large", tokenizer="t5-large")

# Step 2: Generate a paper summary using T5 model
def generate_summary(text):
    # Use the summarizer pipeline to generate the summary
    summary = summarizer(text, max_length=100, min_length=50, do_sample=False)
    return summary[0]['summary_text']
conference_mapping = {
    "kdd": "Knowledge Discovery and Data Mining",
    "neurips": "Conference on Neural Information Processing Systems",
    "cvpr": "Conference on Computer Vision and Pattern Recognition",
    "tmlr": "Transactions on Machine Learning Research",
    "emnlp": "Conference on Empirical Methods in Natural Language Processing"
}

# Step 3: Combine provided keywords and summary for justification generation
def generate_justification(paper_text, topic_keywords, conference_abbr):
    # Select top 10 keywords (use all of them if less than 10)
    selected_keywords = topic_keywords[:10]
    conference_full_name = conference_mapping.get(conference_abbr, "Unknown Conference")


    # Generate paper summary
    summary = generate_summary(paper_text)

    # Generate justification in the desired format
    justification = f"It is classified into {conference_abbr.upper()} ({conference_full_name}) because the work on {summary}, combined with the key themes of {', '.join(selected_keywords)}, aligns well with the themes of {conference_abbr.upper()}. {conference_abbr.upper()} focuses on advancements in {conference_full_name} research, making it an appropriate conference for your research."

    return justification


In [ ]:
df['just']=np.nan

In [ ]:
for i in range(len(df)):
    if(df["confp"][i]!="na"):
    # Get the conference abbreviation for this row (already in lowercase)
    conference_abbr = df["conference"][i]

    # Generate justification for the paper
    justification = generate_justification(df["Abst"][i], df["topic_keywords"][i], conference_abbr)

    # Store the justification in the new column 'just'
    df.at[i, 'just'] = justification
    else:
      df.at[i, 'just'] = "na"

In [ ]:
# Filter the DataFrame starting from index 15
df_filtered = df.loc[15:, ['y_fin', 'confp', 'just']]

# Save the filtered DataFrame to a CSV file
df_filtered.to_csv('output_f.csv', index=False)

print("CSV file has been saved as 'output_filtered.csv'")
